In [ ]:
%load_ext autoreload
%autoreload 2
import sys
from ridge_utils.DataSequence import DataSequence
import pandas as pd
import os
import matplotlib.pyplot as plt
import cortex
import seaborn as sns
from os.path import join
from collections import defaultdict
import numpy as np
import joblib
from tqdm import tqdm
from sasc.config import FMRI_DIR, STORIES_DIR
from neuro.features import qa_questions, feature_spaces
from neuro.config import brain_drive_resps_dir
import neuro.config
from neuro.features.qa_questions import get_questions, get_merged_questions_v3_boostexamples
from neuro.data import story_names, response_utils
from neuro.features.stim_utils import load_story_wordseqs, load_story_wordseqs_huge

In [ ]:
story_names_list = sorted(story_names.get_story_names(
    use_huge=True, train_or_test='test'))
wordseqs = load_story_wordseqs_huge(story_names_list)


class A:
    subject = 'UTS03'
    use_huge = True
    pc_components = 10000


args = A()


def _get_largest_absolute_coefs(_pca, n_pcs=100, n_coefs_per_pc=100):
    idxs_large = set()
    for i in range(n_pcs):
        coefs = np.abs(_pca.components_[i])
        idxs = np.argsort(coefs)[::-1][:]
        idxs_large.update(idxs)
    idxs_large = np.array(list(idxs_large))
    return idxs_large

In [ ]:
story_names_list

In [ ]:
for subject in ['UTS03', 'UTS02', 'UTS01']:
    args.subject = subject
    for story_name in ['fromboyhoodtofatherhood', 'onapproachtopluto', 'wheretheressmoke']:
        ngrams_list = feature_spaces.get_ngrams_list_main(
            wordseqs[story_name], num_trs_context=1)
        ngrams_list = ngrams_list[10:-5]  # apply trim
        args.pc_components = 10000
        _, resp_test, _pca, _scaler_train, _scaler_test = response_utils.get_resps_full(
            args, args.subject, [story_name], [story_name])

        args.pc_components = -1
        _, resp_test_full = response_utils.get_resps_full(
            args, args.subject, [story_name], [story_name])

        idxs_large = _get_largest_absolute_coefs(_pca)

        resp_test_full[:, idxs_large].shape
        resp_selected = np.hstack((resp_test, resp_test_full[:, idxs_large]))

        # temporal alignment
        offset = 2
        resp_selected = resp_selected[offset:, :]
        ngrams_list = ngrams_list[:-offset]

        # apply convolution smoothing filter over axis 0 of resp
        # plt.plot(resp_selected[:, 0])
        conv_filter = np.array([1/3, 1, 1/3])/(5/3)
        resp_selected = np.apply_along_axis(
            lambda m: np.convolve(m, conv_filter, mode='same'), axis=0, arr=resp_selected)
        # plt.plot(resp_selected[:, 0])

        # trim by 1
        resp_selected = resp_selected[1:-1, :]
        ngrams_list = ngrams_list[1:-1]

        assert resp_selected.shape[0] == len(
            ngrams_list), f'{resp_selected.shape[0]} != {len(ngrams_list)}'

        column_names = ['PC' + str(i) for i in range(resp_test.shape[1])] + \
            ['Vox' + str(i) for i in idxs_large]
        df = pd.DataFrame(
            resp_selected, columns=column_names, index=ngrams_list)
        df.to_pickle(f'{subject.lower()}/{story_name}.pkl')